In [1]:
import pandas as pd
import numpy as np
import keras
from numpy import array
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import GRU
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import RepeatVector
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from matplotlib import pyplot as plt
from itertools import chain
import time

In [2]:
MY_DIR=f"{int(time.time())}"

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/scalation/data/master/COVID/CLEANED_35.csv')
print(df.shape)
date = df['date'].tail(391-44)
deaths = df['deathIncrease'].tail(391-44)
sequence=list(deaths)

(391, 18)


In [5]:
def split_sequence(data,steps_in,steps_out):
    X,y=list(),list()
    for i in range(len(data)):
        seq_end=i+steps_in
        end_out=seq_end+steps_out
        if end_out > len(data):
            break
        seq_x, seq_y=data[i:seq_end], data[seq_end:end_out]
        X.append(seq_x)
        y.append(seq_y)
    return array(X),array(y)

In [6]:
steps_in, steps_out = 21 , 14
X, y=split_sequence(sequence,steps_in,steps_out)

In [7]:
features=1
X=X.reshape(X.shape[0],X.shape[1],1)
print(X.shape, y.shape)

(313, 21, 1) (313, 14)


In [9]:
X=X.astype('float32')
y=y.astype('float32')
# Xa and ya are used for GRU, CNN, LSTM.
Xa=X.reshape(X.shape[0],X.shape[1],1)
ya=y.reshape(y.shape[0],y.shape[1],1)
#Xb and yb are used for CNN
Xb=Xa.reshape(Xa.shape[0],Xa.shape[1])
yb=ya.reshape(ya.shape[0],ya.shape[1])
print(Xa.shape,ya.shape, Xb.shape, yb.shape)

(313, 21, 1) (313, 14, 1) (313, 21) (313, 14)


In [48]:
'''
To build a CNN model or hybrid model like CNN-LSTM remove the commented part in below fucntion and set optimizing 
parameters as you wish.
'''
def build_GRU_model(hp):
    model = keras.Sequential()
    #model.add(layers.Conv1D(filters = 64,kernel_size=3,activation='relu', input_shape=(21,1)))
    #model.add(layers.MaxPooling1D(pool_size=2))
    #model.add(layers.Flatten())
    #model.add(layers.RepeatVector(14)) 
    model.add(layers.GRU(hp.Int('Input-units',min_value=32,max_value=256,step=32),activation='relu', input_shape=(21,1)))
    model.add(layers.RepeatVector(14))    
    model.add(layers.GRU(hp.Int('2ndlast-gru',min_value=32,max_value=256,step=32),activation='relu'))  
    model.add(layers.RepeatVector(14))
    model.add(layers.GRU(hp.Int('last-gru',min_value=32,max_value=256,step=32),activation='relu'))    
    model.add(layers.Dense(14))
    model.compile(optimizer='adam',loss='mape',metrics=['accuracy'])
                    
        
    return model

In [49]:
'''
To search we have to set number of trials and trials per execution here.
'''
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=200,
    executions_per_trial=1,
    directory=MY_DIR)

In [50]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
Input-units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
2ndlast-gru (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}


In [51]:
tuner.search(x=xb, y=yb, epochs=200, batch_size=32, validation_split=0.1)

Trial 45 Complete [00h 00m 12s]
val_accuracy: 0.46875

Best val_accuracy So Far: 0.53125
Total elapsed time: 00h 09m 26s
INFO:tensorflow:Oracle triggered exit


In [52]:
'''
The pickle object stores the all the information gathered by tuner and store it in pickle file
'''
import pickle
with open(f"Final CNN tuned {int(time.time())}.pkl","wb") as f:
    pickle.dump(tuner, f)

In [53]:
'''
Using below lines of code we can print the stored pickel object.
'''
tuner = pickle.load(open("Final CNN tuned 1619759188.pkl","rb"))
print(tuner.get_best_hyperparameters()[0].values)


{'Input-units': 224, '2ndlast-gru': 32}


In [36]:
tuner1 = pickle.load(open("CNN tuned 1619142490.pkl","rb"))
print(tuner1.get_best_hyperparameters()[0].values)


{'Input-units': 96, '2ndlast-gru': 96, 'last-gru': 96}


In [37]:
tuner2 = pickle.load(open("ANN tuned 1619024719.pkl","rb"))
print(tuner1.get_best_hyperparameters()[0].values)


{'Input-units': 96, '2ndlast-gru': 96, 'last-gru': 96}
